<a href="https://colab.research.google.com/github/guptagundlapalli/Applied_Data_Analytics/blob/master/Quantize_and_Evaluate_Llama_3_1_Instruct_with_BitsandBytes%2C_AWQ%2C_GPTQ%2C_and_AutoRound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to quantize and evaluate models with BitsandBytes, AWQ, GPTQ, and AutoRound. The evaluation is done with the Evaluation Harness on Arc Challenge and MMLU.

It also benchmarks the models' inference throughput with vLLM.

All these quantization methods run on consumer hardware and won't require a GPU with more than 24 GB of VRAM.


#Quantization

##AutoRound

In [ ]:
!pip install --upgrade transformers auto-round

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

With asymmetric quantization:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from auto_round import AutoRound

bits, group_size, sym = 4, 128, False
autoround = AutoRound(model, tokenizer, bits=bits, group_size=group_size, batch_size=2, seqlen=512, sym=sym, gradient_accumulate_steps=4, device='cuda')
autoround.quantize()
output_dir = "./AutoRound/GPTQ-asym/"
autoround.save_quantized(output_dir)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

2024-08-08 07:28:27 INFO autoround.py L464: using torch.float16 for quantization tuning


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
2024-08-08 07:29:36 INFO autoround.py L1306: quantizing 1/32, model.layers.0
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.45 `position_ids` will be removed and `position_embeddings` will be mandatory.
2024-08-08 07:32:26 INFO autoround.py L1237: quantized 7/7 layers in the block, loss iter 0: 0.000005 -> iter 184: 0.000002
2024-08-08 07:32:31 INFO autoround.py L1306: quantizing 2/32, model.layers.1
2024-08-08 07:35:19 INFO autoround.py L1237: quantized 7/7 layers in the block, loss iter 0: 0.000194 -> iter 33: 0.000100

With symmetric quantization:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from auto_round import AutoRound

bits, group_size, sym = 4, 128, True
autoround = AutoRound(model, tokenizer, bits=bits, group_size=group_size, batch_size=2, seqlen=512, sym=sym, gradient_accumulate_steps=4, device='cuda')
autoround.quantize()
output_dir = "./AutoRound/GPTQ-sym/"
autoround.save_quantized(output_dir)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

2024-08-08 11:02:21 INFO autoround.py L464: using torch.float16 for quantization tuning


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
2024-08-08 11:03:25 INFO autoround.py L1306: quantizing 1/32, model.layers.0
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.45 `position_ids` will be removed and `position_embeddings` will be mandatory.
2024-08-08 11:06:13 INFO autoround.py L1237: quantized 7/7 layers in the block, loss iter 0: 0.000006 -> iter 174: 0.000002
2024-08-08 11:06:18 INFO autoround.py L1306: quantizing 2/32, model.layers.1
2024-08-08 11:09:06 INFO autoround.py L1237: quantized 7/7 layers in the block, loss iter 0: 0.000855 -> iter 45: 0.000235

#GPTQ

In [ ]:
!pip install --upgrade auto-gptq accelerate datasets optimum
!pip install --upgrade transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer
import torch
model_path = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
w = 4 #quantization to 4-bit. Change to 2, 3, or 8 to quantize with another precision

quant_path = 'Meta-Llama-3.1-8B-Instruct-gptq-'+str(w)+'bit'

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
quantizer = GPTQQuantizer(bits=w, dataset="c4", model_seqlen = 2048)
quantized_model = quantizer.quantize_model(model, tokenizer)

quantized_model.save_pretrained(".//GPTQ/"+quant_path, safetensors=True)
tokenizer.save_pretrained("./GPTQ/"+quant_path)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4689: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


('./drive/MyDrive/Quant-Llama-3.1-8B-Instruct/GPTQ/Meta-Llama-3.1-8B-Instruct-gptq-4bit/tokenizer_config.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/GPTQ/Meta-Llama-3.1-8B-Instruct-gptq-4bit/special_tokens_map.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/GPTQ/Meta-Llama-3.1-8B-Instruct-gptq-4bit/tokenizer.json')

#Bitsandbytes



In [ ]:
!pip install --upgrade transformers bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quant_path = 'Meta-Llama-3.1-8B-Instruct-bnb-4bit'
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config
)


model.save_pretrained("./BnB/"+quant_path, safetensors=True)
tokenizer.save_pretrained("./BnB/"+quant_path)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

('./drive/MyDrive/Quant-Llama-3.1-8B-Instruct/BnB/Meta-Llama-3.1-8B-Instruct-bnb-4bit/tokenizer_config.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/BnB/Meta-Llama-3.1-8B-Instruct-bnb-4bit/special_tokens_map.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/BnB/Meta-Llama-3.1-8B-Instruct-bnb-4bit/tokenizer.json')

#AWQ

In [ ]:
!pip install --upgrade autoawq optimum accelerate
!pip install --upgrade transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
quant_path = 'Meta-Llama-3.1-8B-Instruct-awq-4bit'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model with safetensors
model.save_quantized("./AWQ/"+quant_path, safetensors=True)
tokenizer.save_pretrained("./AWQ/"+quant_path)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
AWQ: 100%|██████████| 32/32 [31:16<00:00, 58.66s/it]
Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('./drive/MyDrive/Quant-Llama-3.1-8B-Instruct/AWQ/Meta-Llama-3.1-8B-Instruct-awq-4bit/tokenizer_config.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/AWQ/Meta-Llama-3.1-8B-Instruct-awq-4bit/special_tokens_map.json',
 './drive/MyDrive/Quant-Llama-3.1-8B-Instruct/AWQ/Meta-Llama-3.1-8B-Instruct-awq-4bit/tokenizer.json')

#Evaluation with the Evaluation Harness

In [ ]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install --upgrade auto-gptq accelerate datasets optimum autoawq bitsandbytes aqlm[gpu]
!pip install --upgrade transformers

  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git to /tmp/pip-req-build-zendakuk
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-zendakuk
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit 0571eeb14d4e48aac51956a726c62cd8b382b3d8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached zstandard-0.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Preparing metadata (setup.py) ... done
  Using cache

In [ ]:
!lm_eval --model hf --model_args pretrained=meta-llama/Meta-Llama-3.1-8B-Instruct --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 4 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [29:54<00:00, 49.51it/s]
fatal: not a git repository (or any of the parent directories): .git
2024-08-08:16:12:07,332 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=meta-llama/Meta-Llama-3.1-8B-Instruct), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 4
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.5179|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.5495|±  |0.0145|
|mmlu                                   |      1|none  |      |acc     |↑  |0.6779|±  |0.0038|
| - humanities                          |      1|none  |      |acc     |↑  |0.6346|±  |0.0067|
|  - formal_logic                       |      0|none  |     0|acc     |↑  |0.5000

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-autoround-gptq-4bit-asym --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

100% 311/311 [00:00<00:00, 589.44it/s]
2024-08-08:16:17:39,169 INFO     [task.py:423] Building contexts for mmlu_moral_disputes on rank 0...
100% 346/346 [00:00<00:00, 588.05it/s]
2024-08-08:16:17:39,774 INFO     [task.py:423] Building contexts for mmlu_high_school_world_history on rank 0...
100% 237/237 [00:00<00:00, 583.75it/s]
2024-08-08:16:17:40,192 INFO     [task.py:423] Building contexts for mmlu_high_school_european_history on rank 0...
100% 165/165 [00:00<00:00, 250.68it/s]
2024-08-08:16:17:40,861 INFO     [task.py:423] Building contexts for mmlu_prehistory on rank 0...
100% 324/324 [00:00<00:00, 586.54it/s]
2024-08-08:16:17:41,430 INFO     [task.py:423] Building contexts for arc_challenge on rank 0...
100% 1172/1172 [00:01<00:00, 1013.27it/s]
2024-08-08:16:17:42,658 INFO     [evaluator.py:457] Running loglikelihood requests
Running loglikelihood requests:   0% 0/88855 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will 

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-autoround-gptq-4bit-sym --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [1:13:08<00:00, 20.25it/s]
fatal: not a git repository (or any of the parent directories): .git
2024-08-08:18:52:10,942 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-autoround-gptq-4bit-sym), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.5111|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.5410|±  |0.0146|
|mmlu                                   |      1|none  |      |acc     |↑  |0.6613|±  |0.0038|
| - humanities                          |      1|none  |      |acc     |↑  |0.6147|±  |0.0068|
|  - formal_logic                       |      0|none  |  

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-gptq-4bit --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [1:13:08<00:00, 20.25it/s]
fatal: not a git repository (or any of the parent directories): .git
2024-08-08:20:11:54,488 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-gptq-4bit), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.4983|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.5222|±  |0.0146|
|mmlu                                   |      1|none  |      |acc     |↑  |0.6586|±  |0.0038|
| - humanities                          |      1|none  |      |acc     |↑  |0.6085|±  |0.0068|
|  - formal_logic                       |      0|none  |     0|acc     |

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-awq-4bit --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [51:04<00:00, 28.99it/s] 
fatal: not a git repository (or any of the parent directories): .git
2024-08-08:21:09:41,782 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-awq-4bit), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.5222|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.5418|±  |0.0146|
|mmlu                                   |      1|none  |      |acc     |↑  |0.6597|±  |0.0038|
| - humanities                          |      1|none  |      |acc     |↑  |0.6079|±  |0.0068|
|  - formal_logic                       |      0|none  |     0|acc     |↑ 

In [ ]:
!lm_eval --model hf --model_args pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-bnb-4bit --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [46:08<00:00, 32.09it/s]
fatal: not a git repository (or any of the parent directories): .git
2024-08-08:22:02:22,463 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=kaitchup/Meta-Llama-3.1-8B-Instruct-bnb-4bit), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.5162|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.5435|±  |0.0146|
|mmlu                                   |      1|none  |      |acc     |↑  |0.6574|±  |0.0038|
| - humanities                          |      1|none  |      |acc     |↑  |0.6066|±  |0.0068|
|  - formal_logic                       |      0|none  |     0|acc     |↑  

In [ ]:
!lm_eval --model hf --model_args pretrained=ISTA-DASLab/Llama-3.1-8B-Instruct-AQLM-PV-2Bit-1x16-hf --tasks mmlu,arc_challenge,pawsx --device cuda:0  --num_fewshot 0 --batch_size 8 --output_path ./results/

Running loglikelihood requests: 100% 88855/88855 [36:27<00:00, 40.62it/s]
fatal: not a git repository (or any of the parent directories): .git
2024-08-09:08:24:40,188 INFO     [evaluation_tracker.py:206] Saving results aggregated
hf (pretrained=ISTA-DASLab/Llama-3.1-8B-Instruct-AQLM-PV-2Bit-1x16-hf), gen_kwargs: (None), limit: None, num_fewshot: 0, batch_size: 8
|                 Tasks                 |Version|Filter|n-shot| Metric |   |Value |   |Stderr|
|---------------------------------------|------:|------|-----:|--------|---|-----:|---|-----:|
|arc_challenge                          |      1|none  |     0|acc     |↑  |0.4761|±  |0.0146|
|                                       |       |none  |     0|acc_norm|↑  |0.4846|±  |0.0146|
|mmlu                                   |      1|none  |      |acc     |↑  |0.5892|±  |0.0040|
| - humanities                          |      1|none  |      |acc     |↑  |0.5471|±  |0.0069|
|  - formal_logic                       |      0|none  |     0|ac

##Inference Throughput with vLLM

In [ ]:
!pip install vllm bitsandbytes
!git clone https://github.com/vllm-project/vllm.git
%cd vllm/benchmarks/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.8 MB/s eta 0:00:00
Cloning into 'vllm'...
remote: Enumerating objects: 27197, done.
remote: Counting objects: 100% (4814/4814), done.
remote: Compressing objects: 100% (664/664), done.
remote: Total 27197 (delta 4467), reused 4198 (delta 4149), pack-reused 22383
Receiving objects: 100% (27197/27197), 23.98 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (20432/20432), done.
/content/vllm/benchmarks/vllm/benchmarks


In [ ]:
llms = ["meta-llama/Meta-Llama-3.1-8B-Instruct",
        "kaitchup/Meta-Llama-3.1-8B-Instruct-autoround-gptq-4bit-asym",
        "kaitchup/Meta-Llama-3.1-8B-Instruct-autoround-gptq-4bit-sym",
        "kaitchup/Meta-Llama-3.1-8B-Instruct-gptq-4bit",
        "kaitchup/Meta-Llama-3.1-8B-Instruct-awq-4bit",
        "ISTA-DASLab/Llama-3.1-8B-Instruct-AQLM-PV-2Bit-1x16-hf"
        ]

for llm in llms:
  print("------------------------------------------------------------")
  print("Inference throughput for "+llm)
  !python benchmark_throughput.py --input-len 100 --output-len 250 --max-model-len 512 --device cuda --model {llm}

print("------------------------------------------------------------")
print("Inference throughput for Meta-Llama-3.1-8B-Instruct-bnb-4bit")
!python benchmark_throughput.py --input-len 100 --output-len 250 --max-model-len 512 --device cuda --model meta-llama/Meta-Llama-3.1-8B-Instruct --enforce_eager -q bitsandbytes --load_format bitsandbytes


------------------------------------------------------------
Inference throughput for meta-llama/Meta-Llama-3.1-8B-Instruct
Namespace(backend='vllm', dataset=None, input_len=100, output_len=250, model='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', quantization=None, tensor_parallel_size=1, n=1, use_beam_search=False, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=512, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='cuda', enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto')
INFO 08-09 13:55:56 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=